In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.metrics import *
from tensorflow.keras.regularizers import l2

from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='selu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='selu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'selu' ))
    model.add(Dropout(0.25))
    model.add(Dense(1,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [9]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [11]:
run_experiment(X,y)

Epoch 1/10
29/29 [==============================] - 1s 22ms/step - loss: 9.8499 - accuracy: 0.5203 - precision: 0.4468 - recall: 0.0981 - auc: 0.4965
Epoch 2/10
29/29 [==============================] - 1s 21ms/step - loss: 5.6683 - accuracy: 0.5498 - precision: 0.5222 - recall: 0.4673 - auc: 0.5755
Epoch 3/10
29/29 [==============================] - 1s 21ms/step - loss: 4.0185 - accuracy: 0.6375 - precision: 0.6131 - recall: 0.6145 - auc: 0.7062
Epoch 4/10
29/29 [==============================] - 1s 21ms/step - loss: 3.0671 - accuracy: 0.7831 - precision: 0.7590 - recall: 0.7874 - auc: 0.8632
Epoch 5/10
29/29 [==============================] - 1s 21ms/step - loss: 2.6754 - accuracy: 0.9222 - precision: 0.9085 - recall: 0.9276 - auc: 0.9760
Epoch 6/10
29/29 [==============================] - 1s 21ms/step - loss: 2.3810 - accuracy: 0.9573 - precision: 0.9513 - recall: 0.9579 - auc: 0.9908
Epoch 7/10
29/29 [==============================] - 1s 21ms/step - loss: 2.0824 - accuracy: 0.9441 -

In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), kernel_size=3, activation='selu', input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), kernel_size=3, activation='selu'))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), kernel_size=3, activation='selu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'selu' ))
    model.add(Dropout(0.25))
    model.add(Dense(1,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [13]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [14]:
run_experiment_2(X,y)

Epoch 1/10
29/29 [==============================] - 1s 26ms/step - loss: 26.6807 - accuracy: 0.5312 - precision_5: 0.5000 - recall_5: 0.4907 - auc_5: 0.5257
Epoch 2/10
29/29 [==============================] - 1s 26ms/step - loss: 8.1686 - accuracy: 0.5334 - precision_5: 0.5017 - recall_5: 0.6822 - auc_5: 0.5808
Epoch 3/10
29/29 [==============================] - 1s 26ms/step - loss: 7.1187 - accuracy: 0.5301 - precision_5: 0.4987 - recall_5: 0.4486 - auc_5: 0.5265
Epoch 4/10
29/29 [==============================] - 1s 26ms/step - loss: 6.0463 - accuracy: 0.5674 - precision_5: 0.5355 - recall_5: 0.5818 - auc_5: 0.6143
Epoch 5/10
29/29 [==============================] - 1s 26ms/step - loss: 5.2610 - accuracy: 0.6123 - precision_5: 0.5934 - recall_5: 0.5491 - auc_5: 0.6633
Epoch 6/10
29/29 [==============================] - 1s 26ms/step - loss: 4.6428 - accuracy: 0.6462 - precision_5: 0.6180 - recall_5: 0.6425 - auc_5: 0.7000
Epoch 7/10
29/29 [==============================] - 1s 24ms/ste

In [15]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), kernel_size=3, activation='selu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), kernel_size=3, activation='selu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'selu' ))
    model.add(Dropout(0.25))
    model.add(Dense(50,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'selu' ))
    model.add(Dropout(0.25))
    model.add(Dense(1,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [16]:
def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_3(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [17]:
run_experiment_3(X,y)

Epoch 1/10
29/29 [==============================] - 1s 20ms/step - loss: 10.2507 - accuracy: 0.5794 - precision_10: 0.5531 - recall_10: 0.5350 - auc_10: 0.6100
Epoch 2/10
29/29 [==============================] - 1s 19ms/step - loss: 8.7370 - accuracy: 0.8039 - precision_10: 0.7971 - recall_10: 0.7804 - auc_10: 0.8694
Epoch 3/10
29/29 [==============================] - 1s 19ms/step - loss: 7.0390 - accuracy: 0.9091 - precision_10: 0.8929 - recall_10: 0.9159 - auc_10: 0.9679
Epoch 4/10
29/29 [==============================] - 1s 19ms/step - loss: 5.7507 - accuracy: 0.9617 - precision_10: 0.9645 - recall_10: 0.9533 - auc_10: 0.9852
Epoch 5/10
29/29 [==============================] - 1s 20ms/step - loss: 4.8000 - accuracy: 0.9726 - precision_10: 0.9675 - recall_10: 0.9743 - auc_10: 0.9930
Epoch 6/10
29/29 [==============================] - 1s 20ms/step - loss: 4.2693 - accuracy: 0.9639 - precision_10: 0.9604 - recall_10: 0.9626 - auc_10: 0.9904
Epoch 7/10
29/29 [===========================